<a href="https://colab.research.google.com/github/andromeda123248/Dip/blob/main/SentenceTransformer%20Method.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import numpy as np
import pandas as pd
import math

In [ ]:
! pip install pymorphy2

In [13]:
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import pymorphy2
import re

In [ ]:
import nltk
nltk.download('punkt')

import nltk
nltk.download('stopwords')

In [ ]:
! pip install faiss-gpu ##Installing GPU version of faiss



In [ ]:
! pip install json

In [ ]:
! pip install sentence_transformers ## For textual similarity, using pretrained models

In [ ]:
import faiss
from sentence_transformers import SentenceTransformer, util

Загрузка и подготовка данных

In [ ]:
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')

In [ ]:
json_file_path = 'ex_tr'

df = pd.read_json(json_file_path)

df[['name', 'description', 'genres']] = df['docs'].apply(
        lambda x: pd.Series([x['name'], x['description'], ', '.join([genre['name'] for genre in x['genres']])]))

df = df.drop(columns=['docs'])

In [32]:
df

,name,description,genres,id
0,"Там, где танцуют стерхи",Восьмилетний Ваня переживает смерть матери. Он...,драма,0
1,Паук,Старшеклассника Паркера кусает радиоактивный п...,"короткометражка, ужасы",1
2,Карина,"В конце лета 2014 года, перед наступлением хол...","биография, детектив, драма",2
3,Приговор,2009 год. Небольшой городок на берегу Иссык-Ку...,"драма, триллер",3
4,Джойка,"Джой Уомак — одна из немногих американок, кото...","драма, биография",4
...,...,...,...,...
1495,Невезучие,Перед заключением Руби успевает спрятать награ...,"комедия, криминал",1495
1496,Ангел-А,28-летний Андре должен десятки тысяч евро разл...,"фэнтези, драма, мелодрама, комедия",1496
1497,Сердцеедки,Очаровательная интриганка Макс Коннерс и ее пр...,"мелодрама, комедия, криминал",1497
1498,Апокалипсис сегодня,Во время Вьетнамской войны капитан Уиллард отп...,"военный, драма, история, боевик",1498


In [21]:
#Добавление колонки 'id'
df['id']=df.index

Создание эмбеддингов описаний фильмов

In [22]:
sentences = df['description'].tolist()
print("Number of Sentences in Movie Description ",len(sentences))

Number of Sentences in Movie Description  1500


In [23]:
embeddings = model.encode(sentences)
#Индексирование эмбеддингов с помощью Faiss
faiss.normalize_L2(embeddings)

In [24]:
print("Shape of the EMbeddings is ",embeddings.shape)

Shape of the EMbeddings is  (1500, 768)


In [25]:
dim=768 #размерность эмбеддингов
ncentroids=50 #кол-во центроидов
m=16
quantiser = faiss.IndexFlatL2(dim)
index = faiss.IndexIVFPQ (quantiser, dim,ncentroids, m , 8)
index.train(embeddings)
print(index.is_trained)
faiss.write_index(index, "trained.index")

True


In [26]:
ids = df['id'].tolist()
ids = np.array(ids)
index.add_with_ids(embeddings,ids)
print(index.ntotal)

1500


In [27]:
faiss.write_index(index,"block.index")

Функция поиска похожих фильмов по текстовому запросу

In [28]:
def searchFAISSIndex(data,id_col_name,query,index,nprobe,model,topk=20):
    query_embedding=model.encode([query])[0]
    dim=query_embedding.shape[0]
    query_embedding=query_embedding.reshape(1,dim)
    faiss.normalize_L2(query_embedding)


    index.nprobe=nprobe

    D,I=index.search(query_embedding,topk)
    #Подготовка результатов поиска
    ids=[i for i in I][0]
    L2_score=[d for d in D][0]
    inner_product=[calculateInnerProduct(l2) for l2 in L2_score] #вычисляет косинусную схожесть для каждого расстояния L2
    search_result=pd.DataFrame()
    search_result[id_col_name]=ids
    search_result['cosine_sim']=inner_product
    search_result['L2_score']=L2_score
    #Фильтрация и сортиврока результатов
    dat=data[data[id_col_name].isin(ids)]
    dat=pd.merge(dat,search_result,on=id_col_name)
    dat=dat.sort_values('cosine_sim',ascending=False)
    return dat

In [29]:
def calculateInnerProduct(L2_score):
    return (2-math.pow(L2_score,2))/2

In [30]:
query="история про успешного певца"
search_result=searchFAISSIndex(df,"id",query,index,nprobe=10,model=model,topk=20)
search_result=search_result[['id','description','name','cosine_sim','L2_score']]

In [31]:
search_result

,id,description,name,cosine_sim,L2_score
5,366,"История гитариста Эммета Рэя, о его любовных п...",Сладкий и гадкий,0.867984,0.513841
16,1145,Флоренс Фостер Дженкинс всегда мечтала петь на...,Примадонна,0.840066,0.565570
14,936,"История Иэна Кертиса, фронтмена британской пос...",Контроль,0.828053,0.586424
6,397,История жизни пианиста-виртуоза Дэвид Хельфгот...,Блеск,0.791729,0.645400
13,884,Журналист Стив Лопес встречает Натаниеля Айерс...,Солист,0.761346,0.690874
15,978,У каждого в жизни была минута славы. Или обяза...,Мечты сбываются!,0.758530,0.694939
0,10,Эта музыкальная феерия состоит из самых ярких ...,Бетт Мидлер: Шоу должно продолжаться,0.740203,0.720829
4,277,Замкнутый и одинокий Трэвис берется помогать т...,Стендап по дружбе,0.739379,0.721970
2,81,История жизни известной певицы и активистки фе...,Я — женщина,0.732211,0.731831
11,794,"Стремительный ритм, виртуозная и озорная импро...",Зимний вечер в Гаграх,0.725623,0.740779
